In [3]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import time

In [4]:
# for parallelization if necessary
#from dask import compute, delayed
#from dask.distributed import Client, LocalCluster 
#cl = LocalCluster(n_workers=4)
#client = Client(cl)
#cl

# note: if this throws an error, close jupyter, run "ulimit -n 4096" in terminal, then restart jupyter

In [5]:
# define data location
location = '/Users/mithras/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'
#location = '/Users/loki/Documents/Data/Forecasting Project/'

In [21]:
# read data from pickle

# what data granularity?
granularity = "daily"

# if we're clustering first, we need to load in sufficient_ data to ensure data equivalency across IDs
if granularity == "daily":
    readdata = pd.read_pickle(location+'peco_sufficient_daily.pkl.zip')
    weather = pd.read_pickle(location+'daily_weather.pkl.zip')
    ids = pd.read_pickle(location+'peco_sufficient_daily_ids.pkl.zip')
elif granularity == "hourly":
    readdata = pd.read_pickle(location+'peco_sufficient_hourly.pkl.zip')
    weather = pd.read_pickle(location+'hourly_weather.pkl.zip')
    ids = pd.read_pickle(location+'peco_sufficient_hourly_ids.pkl.zip')
else:
    print("Granularity selected was not 'daily' or 'hourly'")


In [22]:
data = pd.merge(readdata,weather, how="inner", on='Dt')
data = data.drop(columns=['DACCOUNTID','DMETERNO','Dt','DCUSTOMERID','TARIFF','FUELTYPE','REVENUCODE'])
data.head()

,ID,Use,Weekday,Holiday,TYPE,AvgTemp,HighDB,LowDB,Temp1,Temp1delta,...,AvgHumid,Humid1,Humid1delta,AvgWind,Wind1,Wind1delta,AvgDaytimeCloud,AvgCloud,Cloud1,Cloud1delta
0,"(189118224335924, 732846282212)",1.0,1,0,HOME,61.166667,67,57,70.833333,-9.666667,...,78.041667,72.083333,5.958333,13.125,8.833333,4.291667,100.0,92.916667,89.166667,3.75
1,"(8884140853296, 3881418627996)",0.0,1,0,COMM,61.166667,67,57,70.833333,-9.666667,...,78.041667,72.083333,5.958333,13.125,8.833333,4.291667,100.0,92.916667,89.166667,3.75
2,"(224154867687260, 730682241528)",1.0,1,0,HOME,61.166667,67,57,70.833333,-9.666667,...,78.041667,72.083333,5.958333,13.125,8.833333,4.291667,100.0,92.916667,89.166667,3.75
3,"(357349866155308, 792549695076)",0.0,1,0,COMM,61.166667,67,57,70.833333,-9.666667,...,78.041667,72.083333,5.958333,13.125,8.833333,4.291667,100.0,92.916667,89.166667,3.75
4,"(236128365767248, 610644449116)",1.0,1,0,HOME,61.166667,67,57,70.833333,-9.666667,...,78.041667,72.083333,5.958333,13.125,8.833333,4.291667,100.0,92.916667,89.166667,3.75


## Regression ##

In [14]:
# run regression per ID tuple
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from itertools import chain

regr = linear_model.Lasso(alpha = 0.1)

# create table of each cluster's regression equations
rTable = pd.DataFrame(index=(['intercept']+list(home_centroids)[3:] + ['MSE','R2']))


for idx in list(ids):
   # data = readdata[readdata['ID']==idx]
    
    y = data[data['ID']==idx]['Use']
    X = data[data['ID']==idx].drop(columns=['ID','Use'])
    
    

KeyboardInterrupt: 

In [38]:
# run regression per cluster
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from itertools import chain

regr = linear_model.Lasso(alpha = 0.1)

# create table of each cluster's regression equations
rTable = pd.DataFrame(index=(['intercept']+list(home_centroids)[3:] + ['MSE','R2']))

# run regressions; add equation to table
for cluster in home_centroids['cluster'].unique():
    data = home_centroids[(home_centroids['cluster']==cluster)]
    X = data.drop(columns=['cluster','Dt','Use'])
    y = data['Use']
    
    # fit the regression model
    regr.fit(X, y)
    
    # extract model coefficients
    intercept = regr.intercept_
    coefs = regr.coef_
    err = mean_squared_error(y, regr.predict(X))
    r2 = regr.score(X,y)
    #print([intercept]+list(chain(coefs))+[err]+[r2])
    rTable['cluster'+str(cluster)] = [intercept]+list(chain(coefs))+[err]+[r2]

rTable = rTable.T
#rTable.to_csv(location+'daily_home_clusters.csv')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [39]:
rTable

,intercept,AvgTemp,HighDB,LowDB,Temp1,Temp1delta,HighDB1,HighDB1delta,LowDB1,LowDB1delta,...,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Holiday,MSE,R2
cluster0,1.047097e+09,6.356750e+11,-2.453885e+11,1.406305e+10,-6.356750e+11,-6.356750e+11,2.453885e+11,2.453885e+11,-1.406305e+10,-1.406305e+10,...,-1.047097e+09,-1.047097e+09,-1.047097e+09,-1.047097e+09,-1.047097e+09,-1.047097e+09,-1.047097e+09,0.575940,0.381037,0.921426
cluster1,9.603590e+08,5.830176e+11,-2.250612e+11,1.289811e+10,-5.830176e+11,-5.830176e+11,2.250612e+11,2.250612e+11,-1.289811e+10,-1.289811e+10,...,-9.603590e+08,-9.603590e+08,-9.603590e+08,-9.603590e+08,-9.603590e+08,-9.603590e+08,-9.603590e+08,0.531813,0.339723,0.914320
cluster2,1.030559e+09,6.256349e+11,-2.415127e+11,1.384094e+10,-6.256349e+11,-6.256349e+11,2.415127e+11,2.415127e+11,-1.384094e+10,-1.384094e+10,...,-1.030559e+09,-1.030559e+09,-1.030559e+09,-1.030559e+09,-1.030559e+09,-1.030559e+09,-1.030559e+09,0.585872,0.357905,0.925441
cluster3,1.038625e+09,6.305314e+11,-2.434029e+11,1.394926e+10,-6.305314e+11,-6.305314e+11,2.434029e+11,2.434029e+11,-1.394926e+10,-1.394926e+10,...,-1.038625e+09,-1.038625e+09,-1.038625e+09,-1.038625e+09,-1.038625e+09,-1.038625e+09,-1.038625e+09,0.596532,0.372722,0.921372
cluster4,1.005563e+09,6.104602e+11,-2.356549e+11,1.350523e+10,-6.104602e+11,-6.104602e+11,2.356549e+11,2.356549e+11,-1.350523e+10,-1.350523e+10,...,-1.005563e+09,-1.005563e+09,-1.005563e+09,-1.005563e+09,-1.005563e+09,-1.005563e+09,-1.005563e+09,0.554951,0.358263,0.922652
cluster5,1.060996e+09,6.441125e+11,-2.486456e+11,1.424971e+10,-6.441125e+11,-6.441125e+11,2.486456e+11,2.486456e+11,-1.424971e+10,-1.424971e+10,...,-1.060996e+09,-1.060996e+09,-1.060996e+09,-1.060996e+09,-1.060996e+09,-1.060996e+09,-1.060996e+09,0.605616,0.385107,0.920592
cluster6,1.015078e+09,6.162363e+11,-2.378846e+11,1.363301e+10,-6.162363e+11,-6.162363e+11,2.378846e+11,2.378846e+11,-1.363301e+10,-1.363301e+10,...,-1.015078e+09,-1.015078e+09,-1.015078e+09,-1.015078e+09,-1.015078e+09,-1.015078e+09,-1.015078e+09,0.557988,0.371624,0.923444
cluster7,1.002874e+09,6.088279e+11,-2.350247e+11,1.346911e+10,-6.088279e+11,-6.088279e+11,2.350247e+11,2.350247e+11,-1.346911e+10,-1.346911e+10,...,-1.002874e+09,-1.002874e+09,-1.002874e+09,-1.002874e+09,-1.002874e+09,-1.002874e+09,-1.002874e+09,0.649735,0.613880,0.946773
cluster8,2.028211e+07,1.231292e+10,-4.753134e+09,2.723990e+08,-1.231292e+10,-1.231292e+10,4.753134e+09,4.753134e+09,-2.723990e+08,-2.723990e+08,...,-2.028210e+07,-2.028210e+07,-2.028210e+07,-2.028210e+07,-2.028210e+07,-2.028210e+07,-2.028210e+07,0.348327,0.142714,0.544892
cluster9,1.104725e+09,6.706600e+11,-2.588937e+11,1.483703e+10,-6.706600e+11,-6.706600e+11,2.588937e+11,2.588937e+11,-1.483703e+10,-1.483703e+10,...,-1.104725e+09,-1.104725e+09,-1.104725e+09,-1.104725e+09,-1.104725e+09,-1.104725e+09,-1.104725e+09,0.666307,0.387755,0.912650


In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

rTable_scaled = pd.DataFrame(scaler.fit_transform(rTable), columns=rTable.columns)
#rTable_scaled.to_csv(location+'daily_home_clusters_scaled.csv')
rTable_scaled

,intercept,AvgTemp,HighDB,LowDB,Temp1,Temp1delta,HighDB1,HighDB1delta,LowDB1,LowDB1delta,...,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Holiday,MSE,R2
0,0.388504,0.388504,-0.388504,0.388504,-0.388504,-0.388504,0.388504,0.388504,-0.388504,-0.388504,...,-0.388504,-0.388504,-0.388504,-0.388504,-0.388504,-0.388504,-0.388504,0.104074,0.093729,0.316892
1,0.104087,0.104087,-0.104087,0.104087,-0.104087,-0.104087,0.104087,0.104087,-0.104087,-0.104087,...,-0.104087,-0.104087,-0.104087,-0.104087,-0.104087,-0.104087,-0.104087,-0.427972,-0.294914,0.254461
2,0.334274,0.334274,-0.334274,0.334274,-0.334274,-0.334274,0.334274,0.334274,-0.334274,-0.334274,...,-0.334274,-0.334274,-0.334274,-0.334274,-0.334274,-0.334274,-0.334274,0.223834,-0.123870,0.352166
3,0.360722,0.360722,-0.360722,0.360722,-0.360722,-0.360722,0.360722,0.360722,-0.360722,-0.360722,...,-0.360722,-0.360722,-0.360722,-0.360722,-0.360722,-0.360722,-0.360722,0.352359,0.015511,0.316411
4,0.252312,0.252312,-0.252312,0.252312,-0.252312,-0.252312,0.252312,0.252312,-0.252312,-0.252312,...,-0.252312,-0.252312,-0.252312,-0.252312,-0.252312,-0.252312,-0.252312,-0.148991,-0.120508,0.327659
5,0.434077,0.434077,-0.434077,0.434077,-0.434077,-0.434077,0.434077,0.434077,-0.434077,-0.434077,...,-0.434077,-0.434077,-0.434077,-0.434077,-0.434077,-0.434077,-0.434077,0.461880,0.132022,0.309562
6,0.283510,0.283510,-0.283510,0.283510,-0.283510,-0.283510,0.283510,0.283510,-0.283510,-0.283510,...,-0.283510,-0.283510,-0.283510,-0.283510,-0.283510,-0.283510,-0.283510,-0.112377,0.005186,0.334622
7,0.243495,0.243495,-0.243495,0.243495,-0.243495,-0.243495,0.243495,0.243495,-0.243495,-0.243495,...,-0.243495,-0.243495,-0.243495,-0.243495,-0.243495,-0.243495,-0.243495,0.993830,2.284127,0.539574
8,-2.978448,-2.978448,2.978448,-2.978448,2.978448,2.978448,-2.978448,-2.978448,2.978448,2.978448,...,2.978448,2.978448,2.978448,2.978448,2.978448,2.978448,2.978448,-2.640284,-2.148212,-2.991136
9,0.577467,0.577467,-0.577467,0.577467,-0.577467,-0.577467,0.577467,0.577467,-0.577467,-0.577467,...,-0.577467,-0.577467,-0.577467,-0.577467,-0.577467,-0.577467,-0.577467,1.193647,0.156928,0.239789


In [ ]:
# PCA
pca2 = PCA(n_components=2, whiten=True)
X_pca_1cw = pca1.fit_transform(data_scaled)
plt.scatter(X_pca_1cw[:,0], X_pca_1cw[:,1], c=iris.target,
alpha=0.8, s=60, marker='o', edgecolors='white')
plt.show()
pca_2cw.explained_variance_ratio_.sum() 

In [ ]:
# visualize cluster identities using PCA on weight vector

In [ ]:
# visualize rate-code identities using PCA on weight vector

In [ ]:
# append cluster ID to daily/hourly set

In [5]:
# close dask cluster (if used)
#cl.close

<bound method LocalCluster.close of LocalCluster('tcp://127.0.0.1:63002', workers=1, ncores=1)>